In [17]:
#default_exp etsTargetFunction

In [18]:
#hide
import warnings
warnings.simplefilter('ignore')

<h2> EtsTargetFunction </h2>

In [28]:
#export
import math
import warnings
from collections import namedtuple
from functools import partial
from typing import Optional, Dict, Union, Tuple

from numba import jit
import numpy as np
import pandas as pd

from ipynb.fs.full.ets import cpolyroot 
from ipynb.fs.full.etscalc import etscalc 

In [20]:
# Variables from etsTargetFunction.h file

check_params, admissible = False, False 
par = [] 
y = [] 

nstate, errortype, trendtype, seasontype = int(0), int(0), int(0), int(0)
damped = False 

par_noopt = [] 
lower = []
upper = [] 

opt_crit = ""

nmse = int(0)
bounds = ""
m, n = int(0.0), int(0.0)

state = [] 
alpha, beta, gamma, phi = float(0.0), float(0.0), float(0.0), float(0.0)

#float lists 
e = [] 
amse = [] 

lik, objval = float(0.0), float(0.0)
optAlpha, optBeta, optGamma, optPhi, givenAlpha, givenBeta, givenGamma, givenPhi = False, False, False, False, False, False, False, False

In [21]:
class EtsTargetFunction:

    def __init__(self, p_y, p_nstate, p_errrotype, p_trendtype, p_seasontype, p_damped, 
    p_lower, p_upper, p_opt_crit, p_nmse, p_bounds, p_m, p_optAlpha, p_optBeta, p_optGamma,
    p_optPhi, p_givenAlpha, p_givenBeta, p_givenGamma, p_givenPhi, alpha, beta, gamma, phi):
        self.y = p_y
        self.n = self.y.size() 
        self.nstate = p_nstate
        self.errrotype = p_errrotype

        self.trendtype = p_trendtype
        self.seasontype = p_seasontype
        self.damped = p_damped

        self.lower = p_lower
        self.upper = p_upper

        self.opt_crit = p_opt_crit
        self.nmse = p_nmse
        self.bounds = p_bounds

        self.m = p_m

        self.optAlpha = p_optAlpha
        self.optBeta = p_optBeta
        self.optGamma = p_optGamma
        self.optPhi = p_optPhi

        self.givenAlpha = p_givenAlpha
        self.givenBeta = p_givenBeta
        self.givenGamma = p_givenGamma
        self.givenPhi = p_givenPhi

        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.phi = phi

        lik = 0
        objval = 0

        amse = np.zeros(30)
        e = np.zeros(self.n)


        

In [ ]:
def EtsTargetFunction(p_par: float, p_par_length: int):
    equal = True 

    # Check if the parameter configuration has changed, if not, just return 

    if(p_par_length != par.size()):
        equal = False 
    else:
        for j in range(p_par_length):
            if(p_par[j] != par[j]):
                equal = False 
                break

    #if(equal) return 

    par.clear() 

    for j in range(p_par_length):
        par.append(p_par[j])

    j = 0 
    if(optAlpha): 
        j += 1
        alpha = par[j]
    if(optBeta):
        j += 1
        beta = par[j]
    if(optGamma):
        j += 1 
        gamma = par[j]
    if(optPhi):
        j += 1 
        phi = par[j]

    if(check_params):
        objval = -np.Infinity
        return 

    state.clear() 

    for i in range(par.size() - nstate, par.size()):
        state.append(par[i])
    
    # Add extra state
    if(seasontype != 0):

        sum = float(0)

        if(trendtype != 0):
            trendtype = 1 
        else:
            trendtype = 0 

        for i in range(1 + trendtype, nstate):
            sum += state[i]
    
        if(seasontype == 2):
            seasontype = 1 
        else:
            seasontype = 0 
        
        new_state = m * (seasontype) - sum 

        state.append(new_state)

    # check states 
    if(seasontype == 2):
        min = float(-np.Infinity)
        start = int(1)
        if(trendtype != 0): start = 2 

        for i in range(start, state.size()):
            if(state[i] < min):
                min = state[i]
        
        if(min < 0):
            objval = -np.Infinity
            return 

    
    p = state.size() 
    for i in range(p * y.size()):
        state.append(0)
    
    etscalc(y[0], n, state[0], m, errortype, trendtype, seasontype, alpha, beta, gamma, phi, e[0], lik, amse[0], nmse)

    # Avoid perfect fits 
    if(lik < -1e10): lik = -1e10 

    if(math.isnan(lik)): lik = -np.Infinity
    if(math.fabs(lik + 99999) < 1e-7): lik = -np.Infinity
    
    if(opt_crit == "lik"): objval = lik 
    elif(opt_crit == "mse"): objval = amse[0]
    elif(opt_crit == "amse"):

        mean = float(0)
        for i in range(nmse):
            mean += amse[i] / nmse 
        objval = mean 

    elif(opt_crit == "sigma"):
        mean = float(0)
        ne = e.size()

        for i in range(ne):
            mean += e[i] * e[i] / ne 
        objval = mean 
    
    elif(opt_crit == "mae"):
        mean = float(0)
        ne = e.size()

        for i in range(ne):
            mean += math.fabs(e[i]) / ne 
        objval = mean        

In [16]:
def EtsTargetFunction_admissable() -> bool:
    
    if(phi < 0 or phi > 1+1e-8):
        return False 

    # If gamma was set by the user or it is optimized, the bounds need to be enforced
    
    if(optGamma == False and givenGamma == False):
        if(alpha < 1 - 1/phi or alpha > 1 + 1/phi):
            return False

    elif(m > 1): # seasonal model 
        if(optBeta == False and givenBeta == False):
            beta = 0 
        
        # max(1 - 1/phi - alpha, 0)
        
        d = float(1 - 1/phi - alpha) 

        if(d > 0): d = d
        else: d = 0 
        if(gamma < d or gamma > 1 + 1/phi - alpha): return False 
        if(alpha < 1 - 1/phi - gamma*(1 - m + phi + phi * m) / (2 * phi * m)): return False 
        if(beta < -(1 - phi) * (gamma / m + alpha)): return False 

        # End of easy tests. Now use characteristic equations 

        opr = [] 
        opr.append(1)
        opr.append(alpha + beta - phi)

        for i in range(m - 2):
            opr.append(alpha + beta - alpha * phi)

        opr.append(alpha + beta - alpha * phi + gamma - 1)
        opr.append(phi * (1 - alpha - gamma))

        degree = opr.size() - 1  

        opi = [] 
        np.zeros(np.reshape(opi, (-1, opr.size())))

        zeror = [0] * degree 
        zeroi = [0] * degree 

        fail = bool(False) 

        cpolyroot(opr[0], opi[0], degree, zeror[0], zeroi[0], fail)

        max = float(0.0)

        for i in range(zeror.size()):
            abs_val = float(math.sqrt(zeror[i] * zeror[i] + zeroi[i] * zeroi[i]))
            if(abs_val > max):
                max = abs_val 

        if(max > 1+1e-10): return False 

    return True    


        
        

In [23]:
def EtsTargetFunction_check_params() -> bool:
    if(bounds != "admissable"):
        if(optAlpha):
            if(alpha < lower[0] or alpha > upper[0]):
                return False
        if(optBeta):
            if(beta < lower[1] or beta > alpha or beta > upper[1]):
                return False
        if(optPhi):
            if(phi < lower[3] or phi > upper[3]):
                return False 
        if(optGamma):
            if(gamma < lower[2] or gamma > 1 - alpha or gamma > upper[2]):
                return False
    
    if(bounds != "usual"):
        if(EtsTargetFunction_admissable() == False):
            return False
    
    return True